# Optimizing Airline Routes using Graph Anomaly Detection

### 553.602 Research and Design in Applied Mathematics: Data Mining

Team Members: Krutal Patel, Mansi Goel, Chenyu Xie, Nihaar Thakkar   

Subject Area: Transportation Optimization, Transportation Science 

<img src="Images/airline.jpeg" alt= “network” width="500" height="250">


## Research Goals
1. Analyze Southwest route data and preform Exploratory Data Analysis
2. Gather metrics and statistics over time for Node (Airport) and Edge (Route) attributes 
3. Develop a graph object of the Southwest network data for 2019
4. Apply Graph Anomaly detection methods to detect airports and routes that are significant 
5. Construct an optimization method to assign scores to each 
6. Determine a new route model (by removing or adding airports to service or specific routes)

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import geopy.distance as geodist

import warnings
warnings.filterwarnings("ignore")

# Phase I: Querying and Cleaning Data

1. Database Schema
    - Fleet Data: Information on southwest fleet: Unit Cost ($Millions USD) | Total Cost ($Millions USD) 
    - full_quarterly_data_set: southwest routes over time (we will analyze the most recent year 2019 data)
    - Airports2: routes between two airports data, including metrics such as seats, 


2. Filtering and Cleaning Fleet_Data: 
    - obtain data for all aircraft within the Southwest Airline aircraft fleet

3. Filtering and Cleaning SW_routes_2019 data

#### Node (Airport) Attributes / Properties
1. Airport Host City population
2. Departure Delay probability
3. Arrivial Delay probability
4. Cancellation probability
5. Airport Host City (closest metro area)

#### Edge (Trip X -> Y) Attributes / Properties 
1. Airport Code Pairs
2. Aircraft Type: Unit Cost of Aircraft, Average Age for Aircraft on service
3. Total number of departures between these destinations
4. No. of Cancellations between these airports
5. Route demand: total number of passengers 
6. % of Seats filled = passengers on flight / total seats avaliable 
7. Average fuel cost for route/aircraft operation
8. Average stock price in US 2019
9. Distance between airports in miles
10. Total operating expense
11. Baseline revenue 
12. flight Duration (in min)

In [2]:
fleet_data = pd.read_csv("Database/Fleet Data.csv")
# compiled data on 737 statistics for southwest fleets 
sw_737 = fleet_data.loc[fleet_data['Airline'] == 'Southwest Airlines']   # fixed sw_737 data - no changes should be applied here
unit_cost_737 = int(sw_737['Unit Cost'].values[0][1:3])  # dataset 1
avg_age_737 = int(sw_737['Average Age'].values[0])

In [3]:
sw_routes = pd.read_csv("Database/full_quarterly_data_set.csv")
# focus on year 2019 for network generation 
sw_routes_2019 = sw_routes.loc[sw_routes['year'] == 2019]
# compile statistics between the two quarters 
quarters = set(sw_routes_2019['quarter'].to_list())
# split citypair column to origin and destination key id columns 
sw_routes_2019[['Origin_Airport_Code','Destination_Airport_Code']] = sw_routes_2019['citypair'].str.split('-',expand=True)
# In the following code, we extract all of the unique airports serviced by southwest airlines - this will eventually produce a node table for us 
# To compute this, we extract origin and destination airport lists
SW_all_destinations = list(set(sw_routes_2019['Origin_Airport_Code'].to_list() + sw_routes_2019['Destination_Airport_Code'].to_list()))
print('Total no. of Airports Serviced by Southwest Airlines in 2019', len(SW_all_destinations))
# Compile data for the citypair into a list to be able to compare/query from the other Airports2 dataset
citypair_list = list(set(sw_routes_2019['citypair'].to_list()))

# Develop the final edge (airport to airport) route data with attributes 

# For the first attribute--> let's combine together the aircraft fleet information: We need to determine the unqiue aircraft types that Southwest has
list_of_unique_aircrafts_SW = np.unique(sw_routes_2019['aircraft'].to_list())
print(list_of_unique_aircrafts_SW) # these are all 737 family aircraft so the data from above can be applied here
sw_routes_2019['Aircraft_Unit_Cost ($ millions USD)'] = unit_cost_737
# Second attribute to add is the avaerage age of the aircraft that is on service: For this, we will develop a metric s.t. 1/age produces a higher corresponding value.
# This is because, older aircraft tend to me closer to retirement and thus less profitable for airliens as there is usually more mainteance that is needed
sw_routes_2019['Aircraft_Inverse_Age 1/age (1/yrs)'] = 1/avg_age_737

sw_edge_data_fin = sw_routes_2019.groupby('citypair').agg({'sum_departures_performed':'sum','sum_departures_scheduled':'sum', 'passengers':'sum', 'seats':'sum', 
    'avg_fuel_price':'mean', 'avg_stock_price':'mean', 'total_operating_expense':'sum', 'revenue':'sum', 'Aircraft_Unit_Cost ($ millions USD)':'mean', 
    'Aircraft_Inverse_Age 1/age (1/yrs)':'mean'})

# Add new column attributes by apply a transformation using other column attributes, as specified 
sw_edge_data_fin['Number_of_Cancellations'] = sw_edge_data_fin['sum_departures_scheduled'] - sw_edge_data_fin['sum_departures_performed']
sw_edge_data_fin['Proportion of Seats filled'] = sw_edge_data_fin['passengers'] / sw_edge_data_fin['seats']
sw_edge_data_fin['Aircraft Age (yrs)'] = 1 / sw_edge_data_fin['Aircraft_Inverse_Age 1/age (1/yrs)']
sw_edge_data_fin = sw_edge_data_fin.reset_index()
sw_edge_data_fin[['Origin_Airport_Code','Destination_Airport_Code']] = sw_edge_data_fin.citypair.str.split('-',expand=True)


Total no. of Airports Serviced by Southwest Airlines in 2019 85
['Boeing 737-700/700LR/Max 7' 'Boeing 737-800' 'Boeing B737 Max 800']


In [4]:
# Read in the airports dataset  AIRPORTS2
airports_data = pd.read_csv('Database\Airports2.csv')
# This dataset contains A LOT of data records about flight routes between any two destinations both within and outside the mainland United States
# Based on the queried total flight route data from above, we must filter this dataframe so we only look at those specific routes and the corresponding information from them
# Note, the route data we are looking at from the above dataset is 2019-based, we are compiling the data below as initial predictors and attributes 
airports_data['citypair'] = airports_data['Origin_airport'].astype(str) + '-' + airports_data['Destination_airport']
# FIlTER this dataset to contain only the routes extracted from sw_routes_2019
filtered_Airports_Data = airports_data[airports_data['citypair'].isin(citypair_list)]
# Next step is to break apart the dates into date - month - year extractions
# to do this, we first apply a data type transformation to convert col to python dateTime object
filtered_Airports_Data['Fly_date'] =  pd.to_datetime(filtered_Airports_Data['Fly_date'], format='%m/%d/%Y')
# pull out year from fly_date column to filter by year
filtered_Airports_Data['year'] = pd.DatetimeIndex(filtered_Airports_Data['Fly_date']).year  # dataset 3

# Apply grouby methods to combine data by unique flight routes 
filtrd_Airports_aggregated = filtered_Airports_Data.groupby('citypair').agg({'Flights':'sum', 'Distance':'max', 'Org_airport_lat':'max', 
    'Org_airport_long':'max', 'Dest_airport_lat':'max', 'Dest_airport_long':'max'  })

# filter this dataset to extract overlapping airport-route-destination ids
# Notice how there are only 294 intersection iterms in the sets: meaning we will end up with a lot of misisng data. We can cross validate 
# this by reviewing the number of citypair entries from each data
print('No. of Intersected primary keys', len((set(airports_data['citypair'].to_list())).intersection(set(sw_edge_data_fin.index))) )

No. of Intersected primary keys 0


In [5]:
# Thus, we need another dataset to query the relevant datapoints
us_airports = pd.read_csv("Database/us-airports.csv")
# remove uneccesary columns
us_airports = us_airports.drop(labels = ['id','ident','elevation_ft','continent','country_name','iso_country','region_name', 'iso_region', 'local_region', 'keywords',
    'local_code', 'home_link','wikipedia_link','last_updated','scheduled_service','name','gps_code','score'],axis=1)
#us_airports.drop(labels=0,axis=0)
us_airports = us_airports.drop(labels=0,axis=0)
# generate origin table and unque key-value pairs from sw_routes table above
origins = list(set(sw_edge_data_fin['Origin_Airport_Code'].to_list()))
destinations = list(set(sw_edge_data_fin['Destination_Airport_Code'].to_list()))

# us airports contains a key word patch  in iata_code  --> we will inner join the tables
origin_airports = us_airports[us_airports['iata_code'].isin(origins)]
origin_airports = origin_airports.rename(columns={'type': 'origin_airport_type', 'latitude_deg': 'origin_lat', 
    'longitude_deg':'origin_long','iata_code':'Origin_Airport_Code', 'municipality':'origin_city'})

destination_airports = us_airports[us_airports['iata_code'].isin(destinations)]
destination_airports = destination_airports.rename(columns={'type': 'dest_airport_type', 'latitude_deg': 'dest_lat', 
    'longitude_deg':'dest_long','iata_code':'Destination_Airport_Code','municipality':'dest_city'})

# merge origin table with main edge dataset
sw_edge_data_fin = sw_edge_data_fin.merge(origin_airports, how='left')

# apply the same method on destinations dataset
sw_edge_data_fin = sw_edge_data_fin.merge(destination_airports, how='left')


# find the missing airport entry
sw_edge_set = set(sw_edge_data_fin['Origin_Airport_Code'].to_list() + sw_edge_data_fin['Destination_Airport_Code'].to_list())
len(sw_edge_set)
airports_set = set(origin_airports['Origin_Airport_Code'].to_list())
print(sw_edge_set - airports_set)
# SJU contains missing data points- let's fix this by a simple lookup
sju_index = sw_edge_data_fin.loc[sw_edge_data_fin['Origin_Airport_Code'] == 'SJU']
## double check that the nan values only belong to iata code = 'SJU'
df2=sw_edge_data_fin[sw_edge_data_fin.origin_lat.notnull()]
print(len(sju_index) == len(sw_edge_data_fin) - len(sw_edge_data_fin[sw_edge_data_fin.origin_lat.notnull()]))
# Boolean value is True.  This we can apply fillNA to replace the null values for SJU IATA code
sw_edge_data_fin[['origin_airport_type']] = sw_edge_data_fin[['origin_airport_type']].fillna('medium_airport')
sw_edge_data_fin[['dest_airport_type']] = sw_edge_data_fin[['dest_airport_type']].fillna('medium_airport')
sw_edge_data_fin[['dest_city']] = sw_edge_data_fin[['dest_city']].fillna('San Juan')

# edit and replace the origin long and lat coordinates 
sw_edge_data_fin[['origin_lat']] = sw_edge_data_fin[['origin_lat']].fillna('18.4395')
sw_edge_data_fin[['origin_long']] = sw_edge_data_fin[['origin_long']].fillna('-65.9992')
# edit and replace the destination long and lat cooordinates 
sw_edge_data_fin[['dest_lat']] = sw_edge_data_fin[['dest_lat']].fillna('18.4395')
sw_edge_data_fin[['dest_long']] = sw_edge_data_fin[['dest_long']].fillna('-65.9992')
sw_edge_data_fin[['origin_city']] = sw_edge_data_fin[['origin_city']].fillna('San Juan')

# check if values were stored in both cases 

# sw_edge_data_fin.loc[sw_edge_data_fin['Destination_Airport_Code'] == 'SJU']
# sw_edge_data_fin.loc[sw_edge_data_fin['Origin_Airport_Code'] == 'SJU']

# Adding distance metrics 
# define the function call
def distance_calc(origin_lat, origin_long, dest_lat, dest_long):
    if origin_lat == 'nan':
        return 0
    else:
        coords_1 = (origin_lat, origin_long)
        coords_2 = (dest_lat, dest_long)
        dist = geodist.geodesic(coords_1, coords_2).miles
        return dist

sw_edge_data_fin['Route_Distance'] = sw_edge_data_fin.apply(lambda x: distance_calc(x['origin_lat'], x['origin_long'], x['dest_lat'], x['dest_long']), axis=1)

{'SJU'}
True


In [6]:
# we need to now incorporate data from us cities
# let's take a look at the data in the US Cities file
us_cities = pd.read_csv('Database/us_cities.csv')
# let's drop irrelevant columns to reduce data size and to allow for a more efficient merge
us_cities = us_cities.drop(labels = ['city_ascii','state_id','state_name','county_fips','county_name','source',
    'incorporated', 'timezone', 'zips', 'ranking','lat','lng','id'],axis=1)
# The us cities data should contain unique entries so we need to combine the data using group by and get unique city indentifiers
us_cities_aggregated = us_cities.groupby('city').agg({'population':'max', 'density':'max'})
# reset index to get numerical indices 
us_cities_aggregated = us_cities_aggregated.reset_index()
# create copies for origin and destination cities and merge with isin function
origin_cities = list(set(sw_edge_data_fin['origin_city'].to_list()))
destination_cities = list(set(sw_edge_data_fin['dest_city'].to_list()))


origin_cities_tbl = us_cities_aggregated[us_cities_aggregated['city'].isin(origin_cities)]
origin_cities_tbl = origin_cities_tbl.rename(columns={'city': 'origin_city', 'population': 'origin_population', 
    'density':'origin_density'})
dest_cities_tbl = us_cities_aggregated[us_cities_aggregated['city'].isin(destination_cities)]
dest_cities_tbl = dest_cities_tbl.rename(columns={'city': 'dest_city', 'population': 'dest_population', 
    'density':'dest_density'})
# merge all entries on left join
sw_edge_data_fin = sw_edge_data_fin.merge(origin_cities_tbl, how='left')
# apply the same method on destinations dataset
sw_edge_data_fin = sw_edge_data_fin.merge(dest_cities_tbl, how='left')


### Quick Statistics for Southwest Network in 2019

#### 85 Airports Serviced       
#### 1240 Total Flight routes    

In [7]:
#  display the final edge table
# save the final dataset as csv for export
sw_edge_data_fin.to_csv('Database/FINAL_EDGE_TABLE.csv')
sw_edge_data_fin.head()

,citypair,sum_departures_performed,sum_departures_scheduled,passengers,seats,avg_fuel_price,avg_stock_price,total_operating_expense,revenue,Aircraft_Unit_Cost ($ millions USD),...,origin_city,dest_airport_type,dest_lat,dest_long,dest_city,Route_Distance,origin_population,origin_density,dest_population,dest_density
0,ABQ-BWI,108.0,110.0,13736.0,15444.0,57.427544,52.842995,9585665.0,11058,74.0,...,Albuquerque,large_airport,39.1754,-76.668297,Baltimore,1670.237770,765693.0,1155.5,2205092.0,2872.8
1,ABQ-DAL,488.0,504.0,56344.0,74104.0,56.594778,53.008226,14229909.0,16207,74.0,...,Albuquerque,medium_airport,32.847099,-96.851799,Dallas,580.226586,765693.0,1155.5,5668165.0,1522.2
2,ABQ-DEN,330.0,337.0,37782.0,47510.0,56.594778,53.008226,14229909.0,16207,74.0,...,Albuquerque,large_airport,39.86169815,-104.6729965,Denver,349.103284,765693.0,1155.5,2650725.0,1805.7
3,ABQ-HOU,326.0,335.0,39590.0,50650.0,56.927884,52.942134,23815574.0,27265,74.0,...,Albuquerque,medium_airport,29.645399,-95.2789,Houston,759.153724,765693.0,1155.5,5650910.0,1394.6
4,ABQ-LAS,324.0,333.0,35966.0,46332.0,57.427544,52.842995,9585665.0,11058,74.0,...,Albuquerque,large_airport,36.083361,-115.151817,Las Vegas,486.424110,765693.0,1155.5,2150373.0,1754.7


In [8]:
# read in airline on time preformance data
on_time_data = pd.read_csv('Database/August 2018 Nationwide.csv')
# generate new column for the foreign key (route- origin-destination key)
on_time_data['citypair'] = on_time_data['ORIGIN'].astype(str) + '-' + on_time_data['DEST']   # dataset 4
# read in the dataset - DOT airline on time performance statistics
on_time_data_carriers = pd.read_csv('Database/Air Carriers.csv')
# extract corresponding code for southwest airlines
sw_carrier_code = on_time_data_carriers.loc[on_time_data_carriers['Description'] == "Southwest Airlines Co.: WN", 'Code']
print(sw_carrier_code) 

362    19393
Name: Code, dtype: int64


In [9]:
# lets first pull statistics 
# Here, we will treat delay D ~ r.v.  average 
on_time_data

,FL_DATE,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_DELAY,DEP_DELAY_NEW,ARR_DELAY,ARR_DELAY_NEW,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,citypair
0,8/1/2018,1587,12478,JFK,14107,PHX,9.0,9.0,44.0,44.0,0,342,377.0,9.0,0.0,35.0,0.0,JFK-PHX
1,8/1/2018,1588,14107,PHX,11618,EWR,29.0,29.0,53.0,53.0,0,285,309.0,0.0,0.0,53.0,0.0,PHX-EWR
2,8/1/2018,1590,11042,CLE,11298,DFW,-3.0,0.0,-2.0,0.0,0,176,177.0,NaN,NaN,NaN,NaN,CLE-DFW
3,8/1/2018,1591,14843,SJU,11298,DFW,44.0,44.0,43.0,43.0,0,304,303.0,43.0,0.0,0.0,0.0,SJU-DFW
4,8/1/2018,1593,10423,AUS,13303,MIA,-4.0,0.0,-2.0,0.0,0,173,175.0,NaN,NaN,NaN,NaN,AUS-MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701347,8/31/2018,2357,14869,SLC,11298,DFW,-6.0,0.0,-17.0,0.0,0,161,150.0,NaN,NaN,NaN,NaN,SLC-DFW
701348,8/31/2018,2358,15376,TUS,13930,ORD,-10.0,0.0,12.0,12.0,0,204,226.0,NaN,NaN,NaN,NaN,TUS-ORD
701349,8/31/2018,2360,11298,DFW,12892,LAX,2.0,2.0,-14.0,0.0,0,198,182.0,NaN,NaN,NaN,NaN,DFW-LAX
701350,8/31/2018,2361,12892,LAX,13204,MCO,-2.0,0.0,-5.0,0.0,0,307,304.0,NaN,NaN,NaN,NaN,LAX-MCO


To DO next
4. analyze the on time preformance dataset and gather metrics to add to edge table
